In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt


In [25]:
class KarateClubDataset(Dataset):
    def __init__(self):
        self.num_nodes = 34
        self.x = torch.eye(self.num_nodes)  # One-hot node features

        # Define undirected edge list (Zachary's Karate Club)
        edges = np.array([
            [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8],
            [0, 10], [0, 11], [0, 12], [0, 13], [0, 17], [0, 19], [0, 21],
            [0, 31], [1, 2], [1, 3], [1, 7], [1, 13], [1, 17], [1, 19],
            [1, 21], [1, 30], [2, 3], [2, 7], [2, 27], [2, 28], [2, 32],
            [3, 7], [3, 12], [3, 13], [4, 6], [4, 10], [5, 6], [5, 10],
            [5, 16], [6, 16], [8, 30], [8, 32], [8, 33], [9, 33], [13, 33],
            [14, 32], [14, 33], [15, 32], [15, 33], [18, 32], [18, 33],
            [19, 33], [20, 32], [20, 33], [22, 32], [22, 33], [23, 25],
            [23, 27], [23, 29], [23, 32], [23, 33], [24, 25], [24, 27],
            [24, 31], [25, 31], [26, 29], [26, 33], [27, 33], [28, 31],
            [28, 33], [29, 32], [29, 33], [30, 32], [30, 33], [31, 32],
            [31, 33], [32, 33]
        ])

        # Convert to edge_index with both directions (undirected)
        edge_index = np.array(edges).T
        edge_index = np.concatenate((edge_index, edge_index[::-1]), axis=1)
        self.edge_index = torch.tensor(edge_index, dtype=torch.long)

    def __len__(self):
        return 1  # Only one graph

    def __getitem__(self, idx):
        return self.x, self.edge_index



In [26]:
dataset = KarateClubDataset()
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# For testing
for x, edge_index in loader:
    print("Node features shape:", x.shape)
    print("Edge index shape:", edge_index.shape)


Node features shape: torch.Size([1, 34, 34])
Edge index shape: torch.Size([1, 2, 150])


In [27]:
class GCNConv(torch.nn.Module):
    def __init__(self, num_features, output_dim):
        super().__init__()
        self.W0 = torch.nn.Parameter(torch.randn(num_features, 16))
        self.W1 = torch.nn.Parameter(torch.randn(16, output_dim))

    def g_conv(self, x, w, edge_indices):
        num_nodes = x.size(0)
        A = torch.zeros((num_nodes, num_nodes), device=x.device)
        
        A[edge_indices[0], edge_indices[1]] = 1
        A += torch.eye(num_nodes, device=x.device)  # Add self-connections
        D_inv_sqrt = torch.diag(torch.pow(A.sum(1), -0.5))
        A_hat = D_inv_sqrt @ A @ D_inv_sqrt
        return A_hat @ x @ w

    def forward(self, x, edge_index):
        h1 = self.g_conv(x, self.W0, edge_index).relu()
        h = self.g_conv(h1, self.W1, edge_index).softmax(dim=1)
        return h


In [28]:
# Create model
model = GCNConv(num_features=34, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Example: dummy labels (use real labels if available)
labels = torch.randint(0, 2, (34,))

# Training loop
for x, edge_index in loader:
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(x.squeeze(0), edge_index.squeeze(0))
        loss = loss_fn(out, labels)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f}")


Epoch   0 | Loss: 0.7374
Epoch  10 | Loss: 0.6937
Epoch  20 | Loss: 0.6506
Epoch  30 | Loss: 0.6191
Epoch  40 | Loss: 0.5923
Epoch  50 | Loss: 0.5700
Epoch  60 | Loss: 0.5499
Epoch  70 | Loss: 0.5315
Epoch  80 | Loss: 0.5145
Epoch  90 | Loss: 0.4987
Epoch 100 | Loss: 0.4838
Epoch 110 | Loss: 0.4701
Epoch 120 | Loss: 0.4575
Epoch 130 | Loss: 0.4455
Epoch 140 | Loss: 0.4332
Epoch 150 | Loss: 0.4204
Epoch 160 | Loss: 0.4091
Epoch 170 | Loss: 0.3996
Epoch 180 | Loss: 0.3917
Epoch 190 | Loss: 0.3851
